In [1]:
import sys 
import pytorch_lightning as pl
import seml
import torch
from sacred import SETTINGS, Experiment

sys.path.insert(0,"../")
from paths import EXPERIMENT_FOLDER

from scCFM.datamodules.sc_datamodule import scDataModule
from scCFM.models.base.vae import VAE, AE
from scCFM.models.base.geodesic_ae import GeodesicAE

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger 

/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [2]:
config = {
    "training": {
        "task_name": "geom_vae_lib",
        "seed": 42
    },
    "datamodule": {'path': '/nfs/homedirs/pala/scCFM/project_dir/data/eb/processed/eb_phate.h5ad',
                   'x_layer': 'X_norm', 
                   'cond_keys': 'experimental_time', 
                   'use_pca': False,
                   'n_dimensions': None, 
                   'train_val_test_split': [0.8, 0.2],
                   'num_workers': 2, 
                   'batch_size': 32},
    
    "model": {'hidden_dims': [256, 10],
              'batch_norm': True, 
              'dropout': False,
              'dropout_p': False,
              'likelihood': 'nb', 
              'learning_rate': 0.001
              },
    
    "model_checkpoint": {'filename': 'epoch_{epoch:04d}', 
                         'monitor': 'val/lik',
                         'mode': 'min', 
                         'save_last': True, 
                         'auto_insert_metric_name': False},
    
    "early_stopping": {'perform_early_stopping': False, 
                       'monitor': 'val/loss', 
                       'patience': 50, 
                       'mode': 'min', 
                       'min_delta': 0.0,
                       'verbose': False, 
                       'strict': True,
                       'check_finite': True, 
                       'stopping_threshold': None, 
                       'divergence_threshold': None,
                       'check_on_train_epoch_end': None},
    
    "logger": {'offline': False,
               'id': None,
               'project': 'geom_vae_lib_eb_reg',
               'log_model': True,
               'prefix': '', 
               'group': '', 
               'tags': [], 
               'job_type': ''},
    
    "trainer": {
        "max_epochs": 1000,
        "accelerator": "gpu",
        "devices": 1,
        "log_every_n_steps": 10
    }
}


In [3]:
# Training configuration
# Initialize task_name
task_name = config["training"]["task_name"]

# Fix seed for reproducibility
seed = config["training"]["seed"]
torch.manual_seed(seed)
if seed:
    pl.seed_everything(seed, workers=True)

# Initialize folder
current_experiment_dir = EXPERIMENT_FOLDER / task_name
current_experiment_dir.mkdir(parents=True, exist_ok=True)

# Datamodule initialization
datamodule = scDataModule(
    path=config["datamodule"]["path"],
    x_layer=config["datamodule"]["x_layer"],
    cond_keys=config["datamodule"]["cond_keys"],
    use_pca=config["datamodule"]["use_pca"],
    n_dimensions=config["datamodule"]["n_dimensions"],
    train_val_test_split=config["datamodule"]["train_val_test_split"],
    batch_size=config["datamodule"]["batch_size"],
    num_workers=config["datamodule"]["num_workers"]
)

# Model initialization
hidden_dims = config["model"]["hidden_dims"]
batch_norm = config["model"]["batch_norm"]
dropout = config["model"]["dropout"]
dropout_p = config["model"]["dropout_p"]
likelihood = config["model"]["likelihood"]
learning_rate = config["model"]["learning_rate"]

vae_kwargs = dict( 
    in_dim=datamodule.in_dim,
    hidden_dims=hidden_dims,
    batch_norm=batch_norm,
    dropout=dropout,
    dropout_p=dropout_p,
    likelihood=likelihood,
    learning_rate=learning_rate)

model = GeodesicAE(**vae_kwargs)

# Model checkpoint initialization
model_ckpt_callbacks = ModelCheckpoint(
    dirpath=current_experiment_dir / "checkpoints",
    filename=config["model_checkpoint"]["filename"],
    monitor=config["model_checkpoint"]["monitor"],
    mode=config["model_checkpoint"]["mode"],
    save_last=config["model_checkpoint"]["save_last"],
    auto_insert_metric_name=config["model_checkpoint"]["auto_insert_metric_name"]
)

# Early stopping initialization
early_stopping_callbacks = None

if config["early_stopping"]["perform_early_stopping"]:
    early_stopping_callbacks = EarlyStopping(
        monitor=monitor_early_stopping,
        patience=config["early_stopping"]["patience"],
        mode=mode_early_stopping,
        min_delta=config["early_stopping"]["min_delta"],
        verbose=config["early_stopping"]["verbose"],
        strict=config["early_stopping"]["strict"],
        check_finite=config["early_stopping"]["check_finite"],
        stopping_threshold=config["early_stopping"]["stopping_threshold"],
        divergence_threshold=config["early_stopping"]["divergence_threshold"],
        check_on_train_epoch_end=config["early_stopping"]["check_on_train_epoch_end"]
    )

# Logger initialization
logger = WandbLogger(
    save_dir=current_experiment_dir,
    offline=config["logger"]["offline"],
    id=config["logger"]["id"],
    project=config["logger"]["project"],
    log_model=config["logger"]["log_model"],
    prefix=config["logger"]["prefix"],
    group=config["logger"]["group"],
    tags=config["logger"]["tags"],
    job_type=config["logger"]["job_type"]
)

# Trainer initialization
if early_stopping_callbacks:
    callbacks = [model_ckpt_callbacks, early_stopping_callbacks]
else:
    callbacks = [model_ckpt_callbacks]

trainer = Trainer(
    callbacks=callbacks,
    default_root_dir=current_experiment_dir,
    logger=logger,
    max_epochs=config["trainer"]["max_epochs"],
    accelerator=config["trainer"]["accelerator"],
    devices=config["trainer"]["devices"],
    log_every_n_steps=config["trainer"]["log_every_n_steps"]
)

[rank: 0] Global seed set to 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: allepalma. Use `wandb login --relogin` to force relogin


/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model=model, train_dataloaders=datamodule.train_dataloader(), val_dataloaders=datamodule.val_dataloader())

/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python ...
  rank_zero_warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /nfs/students/pala/scCFM/experiments/geom_vae_lib/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/pytorch_lightning/core/optimizer.py:263: RuntimeWarning: Found unsupported keys in the lr scheduler dict: {'min_lr', 'threshold'}. HINT: remove them from the output of

Sanity Checking: 0it [00:00, ?it/s]

/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The

Training: 0it [00:00, ?it/s]

/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (32, 1241) is expensive. Consider setting n_pca.
  warnings.warn(
/nfs/staff-ssd/pala/miniconda3/envs/scCFM/lib/python3.10/site-packages/graphtools/graphs.py:122: UserWarning: Building a kNNGraph on data of shape (